## Model build

In [1]:
import tensorflow as tf
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import os
from torch import tensor
from torchmetrics.audio import ScaleInvariantSignalDistortionRatio
import concurrent.futures
import tqdm

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D,Conv2DTranspose, LeakyReLU, Dropout, concatenate, Multiply,ReLU
from tensorflow.keras.models import Model

In [3]:
if tf.config.list_physical_devices('GPU'):
    print("Using GPU for Training")
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
else:
    print("Using CPU for Training")

Using GPU for Training


## attnunet

In [4]:
# import tensorflow as tf
# import tensorflow.keras.backend as K

# #definig the convolution block that consist of 2 conv layers.

# def conv_block(input,no_filter,f_size,BN = False,drop_out = 0):
#     x = tf.keras.layers.Conv2D(no_filter,f_size, padding = "same")(input)
#     x = LeakyReLU(alpha=0.2)(x)
#     if BN:
#         x = tf.keras.layers.BatchNormalization()(x)
    
#     residual = x
#     x = tf.keras.layers.Conv2D(no_filter,f_size,padding = "same")(x)
#     x = LeakyReLU(alpha=0.2)(x)
    
#     if BN :
#         x = tf.keras.layers.BatchNormalization()(x)
    
#     if drop_out > 0 :
#         x = tf.keras.layers.Dropout(drop_out)(x)
#     x = tf.keras.layers.Add()([x, residual])
#     return x


# def att_block(x, g, desired_dimensionality):
#     x_shape = x.shape
#     g_shape = g.shape
    
    
#     # strides for xl should be 2 to equal the shapes before addition
#     xl = tf.keras.layers.Conv2D(desired_dimensionality, (1, 1),strides=(2,2), padding="same")(x)
#     xl = LeakyReLU(alpha=0.2)(xl)
    
#     gl = tf.keras.layers.Conv2D(desired_dimensionality, (1, 1), padding="same")(g)
#     gl = LeakyReLU(alpha=0.2)(gl)
    
#     xg = tf.keras.layers.Add()([xl, gl])
#     xg = LeakyReLU(alpha=0.2)(xg)
#     xg = tf.keras.layers.Conv2D(1, (1, 1), activation="sigmoid", padding="same")(xg)
#     xg_shape = xg.shape
#     # Transpose xg to match the shape of x
#     xg = tf.keras.layers.Conv2DTranspose(1, (x_shape[1], x_shape[2]), strides=(x_shape[1] // xg_shape[1], x_shape[2] // xg_shape[2]), padding='same')(xg)
    
#     output = tf.keras.layers.Multiply()([xg, x])
#     return output


# def att_model(no_filter, input_shape=(224, 224, 3)):
#     # down-sampling process
#     inputs = tf.keras.layers.Input(input_shape, dtype=tf.float32)
#     x1 = conv_block(inputs, no_filter, (5,5), BN=True, drop_out=0.5)
#     pool1 = tf.keras.layers.MaxPooling2D(2, 2)(x1)

#     x2 = conv_block(pool1, 2 * no_filter, (5,5), BN=True, drop_out=0.5)
#     pool2 = tf.keras.layers.MaxPooling2D(2, 2)(x2)

#     x3 = conv_block(pool2, 4 * no_filter, (5,5), BN=True, drop_out=0.5)
#     pool3 = tf.keras.layers.MaxPooling2D(2, 2)(x3)

#     x4 = conv_block(pool3, 8 * no_filter, (5,5), BN=True, drop_out=0.5)
#     pool4 = tf.keras.layers.MaxPooling2D(2, 2)(x4)

#     # bottle-neck
#     x5 = conv_block(pool4, 16 * no_filter, (5,5), BN=True, drop_out=0.5)

#     # up-sampling layers

#     x6 = att_block(x4, x5, no_filter * 2)
#     u6 = tf.keras.layers.Conv2DTranspose(8 * no_filter,(5,5),strides=(2,2) ,padding='same')(x5)
#     concate1 = tf.keras.layers.Concatenate()([x6, u6])
#     conv6 = conv_block(concate1, 8 * no_filter, (5,5), BN=True, drop_out=0.5)
    
#     x7= att_block(x3,conv6, no_filter*2)
#     u7= tf.keras.layers.Conv2DTranspose(4 * no_filter,(5,5),strides=(2,2), padding='same')(conv6) 
#     concate2 = tf.keras.layers.Concatenate()([x7,u7])
#     conv7 = conv_block(concate2,4*no_filter,(5,5),BN = True,drop_out = 0.5) 


#     x8= att_block(x2,conv7, no_filter*2)
#     u8= tf.keras.layers.Conv2DTranspose(2 * no_filter,(5,5),strides=(2,2), padding='same')(conv7) 
#     concate3 = tf.keras.layers.Concatenate()([x8,u8])
#     conv8 = conv_block(concate3,2*no_filter,(5,5),BN = True,drop_out = 0.5) 



#     x9= att_block(x1,conv8, no_filter*2)
#     u9= tf.keras.layers.Conv2DTranspose(no_filter,(5,5),strides=(2,2),padding='same')(conv8) 
#     concate4 = tf.keras.layers.Concatenate()([x9,u9])
#     conv9 = conv_block(concate4,no_filter,(5,5),BN = True,drop_out = 0.5) 
    


#     conv_final = tf.keras.layers.Conv2D(1, kernel_size=(4,4),dilation_rate=(2, 2), strides=(1,1),activation='sigmoid', padding='same')(conv8)
#     conv_final =tf.keras.layers.BatchNormalization(axis=3)(conv_final)
#     conv_final = tf.keras.layers.Conv2DTranspose(1,(5,5),strides=(2,2), activation="sigmoid", padding='same')(conv_final)
    
#     output = Multiply()([conv_final, inputs])

#     return tf.keras.Model(inputs, output)



# model = att_model(16,input_shape=(128,64,1) )
# model.summary()

# Real Value Unet

In [5]:
# from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, concatenate, Multiply, Activation
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Activation, Add, Multiply

# def attention_block_2d(x, g, inter_channel, data_format='channels_last'):
#     # theta_x(?,g_height,g_width,inter_channel)
#     theta_x = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(x)

#     # phi_g(?,g_height,g_width,inter_channel)
#     phi_g = Conv2D(inter_channel, (1, 1), strides=(1, 1), padding='same')(g)

#     # f(?,g_height,g_width,inter_channel)
#     f = Activation('relu')(Add()([theta_x, phi_g]))

#     # psi_f(?,g_height,g_width,1)
#     psi_f = Conv2D(1, (1, 1), strides=(1, 1), padding='same')(f)
#     rate = Activation('sigmoid')(psi_f)

#     # rate(?,x_height,x_width)
#     # att_x(?,x_height,x_width,x_channel)
#     att_x = Multiply()([x, rate])

#     return att_x

# def unet_with_attention(input_shape):
#     # Encoder
#     inputs = Input(input_shape)
#     conv1 = Conv2D(16, (5, 5), strides=(2, 2), padding='same')(inputs)
#     conv1 = LeakyReLU(alpha=0.2)(conv1)
#     conv1 = Dropout(0.5)(conv1)

#     conv2 = Conv2D(32, (5, 5), strides=(2, 2), padding='same')(conv1)
#     conv2 = LeakyReLU(alpha=0.2)(conv2)
#     conv2 = Dropout(0.5)(conv2)

#     conv3 = Conv2D(64, (5, 5), strides=(2, 2), padding='same')(conv2)
#     conv3 = LeakyReLU(alpha=0.2)(conv3)
#     conv3 = Dropout(0.5)(conv3)

#     conv4 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')(conv3)
#     conv4 = LeakyReLU(alpha=0.2)(conv4)
#     conv4 = Dropout(0.5)(conv4)

#     conv5 = Conv2D(256, (5, 5), strides=(2, 2), padding='same')(conv4)
#     conv5 = LeakyReLU(alpha=0.2)(conv5)
#     conv5 = Dropout(0.5)(conv5)
    
#     #bottleneck
#     conv6 = Conv2D(512, (5, 5), strides=(2, 2), padding='same')(conv5)
#     conv6 = LeakyReLU(alpha=0.2)(conv6)
#     conv6 = Dropout(0.5)(conv6)

#     # Decoder with Attention
#     up1 = Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same')(conv6)
#     up1 = LeakyReLU(alpha=0.2)(up1)
#     attention1 = attention_block_2d(up1, conv5, 128)
#     merge1 = concatenate([conv5, attention1], axis=3)
#     merge1 = Dropout(0.5)(merge1)

#     up2 = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same')(merge1)
#     up2 = LeakyReLU(alpha=0.2)(up2)
#     attention2 = attention_block_2d(up2, conv4, 64)
#     merge2 = concatenate([conv4, attention2], axis=3)
#     merge2 = Dropout(0.5)(merge2)

#     up3 = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same')(merge2)
#     up3 = LeakyReLU(alpha=0.2)(up3)
#     attention3 = attention_block_2d(up3, conv3, 32)
#     merge3 = concatenate([conv3, attention3], axis=3)
#     merge3 = Dropout(0.5)(merge3)

#     up4 = Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same')(merge3)
#     up4 = LeakyReLU(alpha=0.2)(up4)
#     attention4 = attention_block_2d(up4, conv2, 16)
#     merge4 = concatenate([conv2, attention4], axis=3)
#     merge4 = Dropout(0.5)(merge4)

#     up5 = Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same')(merge4)
#     up5 = LeakyReLU(alpha=0.2)(up5)
#     attention5 = attention_block_2d(up5, conv1, 8)
#     merge5 = concatenate([conv1, attention5], axis=3)
#     merge5 = Dropout(0.5)(merge5)

#     # Add the final convolution layer
#     conv = Conv2D(1, (4, 4), dilation_rate=(2, 2), activation='sigmoid', padding='same')(merge5)

#     output = Conv2DTranspose(1, (5, 5), strides=(2, 2), activation="sigmoid", padding='same')(conv)
#     output = Multiply()([output, inputs])

#     model = Model(inputs=inputs, outputs=output)
#     return model

# # Define the input shape (adjust as needed)
# input_shape = (128, 64, 1)  # Adjusted to (128, 128, 1)

# # Create the U-Net model with attention
# model_with_attention = unet_with_attention(input_shape)

# # Print the model summary
# model_with_attention.summary()


In [6]:
def unet(input_shape):
    # Encoder
    inputs = Input(input_shape)
    conv1 = Conv2D(16, (3,3), strides=(2, 2), padding='same')(inputs)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = Dropout(0.5)(conv1)

    conv2 = Conv2D(32, (3,3), strides=(2, 2), padding='same')(conv1)
    conv2 = LeakyReLU(alpha=0.2)(conv2)
    conv2 = Dropout(0.5)(conv2)

    conv3 = Conv2D(64, (3,3), strides=(2, 2), padding='same')(conv2)
    conv3 = LeakyReLU(alpha=0.2)(conv3)
    conv3 = Dropout(0.5)(conv3)

    conv4 = Conv2D(128, (3,3), strides=(2, 2), padding='same')(conv3)
    conv4 = LeakyReLU(alpha=0.2)(conv4)
    conv4 = Dropout(0.5)(conv4)

    conv5 = Conv2D(256, (3,3), strides=(2, 2), padding='same')(conv4)
    conv5 = LeakyReLU(alpha=0.2)(conv5)
    conv5 = Dropout(0.5)(conv5)

    conv6 = Conv2D(512, (3,3), strides=(2, 2), padding='same')(conv5)
    conv6 = LeakyReLU(alpha=0.2)(conv6)
    conv6 = Dropout(0.5)(conv6)

    # Decoder
    up1 = Conv2DTranspose(256, (3,3), strides=(2, 2), padding='same')(conv6)
    up1 = LeakyReLU(alpha=0.2)(up1)
    merge1 = concatenate([conv5, up1], axis=3)  # Changed axis to concatenate along the channel dimension
    merge1 = Dropout(0.5)(merge1)

    up2 = Conv2DTranspose(128, (3,3), strides=(2, 2), padding='same')(merge1)
    up2 = LeakyReLU(alpha=0.2)(up2)
    merge2 = concatenate([conv4, up2], axis=3)  # Changed axis to concatenate along the channel dimension
    merge2 = Dropout(0.5)(merge2)

    up3 = Conv2DTranspose(64, (3,3), strides=(2, 2), padding='same')(merge2)
    up3 = LeakyReLU(alpha=0.2)(up3)
    merge3 = concatenate([conv3, up3], axis=3)  # Changed axis to concatenate along the channel dimension
    merge3 = Dropout(0.5)(merge3)

    up4 = Conv2DTranspose(32, (3,3), strides=(2, 2), padding='same')(merge3)
    up4 = LeakyReLU(alpha=0.2)(up4)
    merge4 = concatenate([conv2, up4], axis=3)  # Changed axis to concatenate along the channel dimension
    merge4 = Dropout(0.5)(merge4)

    up5 = Conv2DTranspose(16, (3,3), strides=(2, 2), padding='same')(merge4)
    up5 = LeakyReLU(alpha=0.2)(up5)
    merge5 = concatenate([conv1, up5], axis=3)  # Changed axis to concatenate along the channel dimension
    merge5 = Dropout(0.5)(merge5)

    # Add the final convolution layer
    conv = Conv2D(1, (4, 4), dilation_rate=(2, 2), activation='sigmoid', padding='same')(merge5)

    output = Conv2DTranspose(1, (3,3), strides=(2, 2), activation="sigmoid", padding='same')(conv)
    output = Multiply()([output, inputs])

    model = Model(inputs=inputs, outputs=output)
    return model

# Define the input shape (adjust as needed)
input_shape = (128, 64, 1)  # Adjusted to (128, 128, 1)

# Create the U-Net model
model = unet(input_shape)

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 64, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 64, 32, 16)   416         ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 64, 32, 16)   0           ['conv2d[0][0]']                 
                                                                                                  
 dropout (Dropout)              (None, 64, 32, 16)   0           ['leaky_re_lu[0][0]']        

 spose)                                                                                           
                                                                                                  
 leaky_re_lu_10 (LeakyReLU)     (None, 64, 32, 16)   0           ['conv2d_transpose_4[0][0]']     
                                                                                                  
 concatenate_4 (Concatenate)    (None, 64, 32, 32)   0           ['dropout[0][0]',                
                                                                  'leaky_re_lu_10[0][0]']         
                                                                                                  
 dropout_10 (Dropout)           (None, 64, 32, 32)   0           ['concatenate_4[0][0]']          
                                                                                                  
 conv2d_6 (Conv2D)              (None, 64, 32, 1)    513         ['dropout_10[0][0]']             
          

## Data Prepare

In [7]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_clean']  # Replace with the actual variable name

# Define the root folder containing subfolders with .mat files
root_folder = 'data/dataformodel_noaug/'

# Specify subfolders of interest
subfolders = [
#     'clean_stft_pink-5db',
#     'clean_stft_pink0db',
#     'clean_stft_pink5db',
    'clean_stft_wgn5db',
    'clean_stft_wgn-5db',
    'clean_stft_wgn0db',
#     'clean_stft_han',
#    'clean_stft_red-5db_Hristo',
#     'clean_stft_red0db_Hristo',
#     'clean_stft_red5db_Hristo',
]

# Initialize an empty list to store loaded data
loaded_data = []

# Loop through each subfolder and load data
for subfolder in subfolders:
    mat_folder = os.path.join(root_folder, subfolder)

    # List all .mat files in the current subfolder
    mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

    # Use concurrent processing to load data from multiple files simultaneously
    with concurrent.futures.ThreadPoolExecutor() as executor:
        current_loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files), desc=f'Loading {subfolder}'))

    # Extend the loaded_data list with data from the current subfolder
    loaded_data.extend(current_loaded_data)

# Stack all loaded data into a single array
clean_data = np.stack(loaded_data)

Loading clean_stft_wgn5db: 4213it [00:02, 1874.03it/s]
Loading clean_stft_wgn-5db: 4213it [00:02, 1752.94it/s]
Loading clean_stft_wgn0db: 4213it [00:02, 1895.01it/s]


In [8]:
clean_data.shape,np.std(np.real(clean_data)),np.mean(np.real(clean_data))

((12639, 128, 64), 0.8289681539628652, 0.0003554751293048475)

In [4]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_clean']  # Replace with the actual variable name

# Define the folder containing the .mat files
mat_folder = 'data/dataformodel_noaug/clean_stft_wgn-5db'  # Replace with the actual path

# Initialize an empty list to store loaded data
loaded_data = []

# List all .mat files in the folder
mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# Use concurrent processing to load data from multiple files simultaneously
with concurrent.futures.ThreadPoolExecutor() as executor:
    loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files)))

# Concatenate all loaded data into a single variable
clean_data = np.stack(loaded_data, axis=0)

4213it [00:02, 1752.20it/s]


In [5]:
clean_data

array([[[-1.01583342e-02+0.00000000e+00j,
          1.48124459e-04+0.00000000e+00j,
         -1.31348991e-01+0.00000000e+00j, ...,
         -9.67382393e-04+0.00000000e+00j,
          6.20926666e-02+0.00000000e+00j,
          1.19025707e-01+0.00000000e+00j],
        [ 1.08736900e-02-6.97484940e-03j,
         -4.14627525e-02+2.75162301e-02j,
          2.29003247e-02+1.82192345e-01j, ...,
          9.60196572e-03+1.82953608e-02j,
         -4.74627357e-02-4.42160033e-02j,
          1.56069789e-03-1.18564067e-01j],
        [-7.23952182e-03+3.09935212e-02j,
          1.14322196e-01+1.62646181e-02j,
          1.41347812e-01-2.25926274e-01j, ...,
          5.05237976e-02+2.00661526e-03j,
          2.91857956e-03+6.23733478e-02j,
         -1.15884004e-01-1.02860766e-02j],
        ...,
        [ 6.40714388e-05-4.23346490e-04j,
         -4.11609985e-04+1.16640631e-03j,
          9.57692710e-04-2.21065042e-03j, ...,
         -1.44351935e-04+1.58730087e-04j,
          1.49992891e-04-1.75334763e-04j

In [10]:
# clean_data.shape,np.std(np.real(clean_data)),np.mean(np.real(clean_data))

In [11]:
clean_real = np.real(clean_data)
clean_img  = np.imag(clean_data)
clean_real.shape,clean_img.shape

((12639, 128, 64), (12639, 128, 64))

In [12]:
# from sklearn.preprocessing import StandardScaler

# original_shape_clean = clean_real.shape
# clean_data_2d = clean_real.reshape(-1, original_shape_clean[-1])

# # Initialize the StandardScaler
# scaler_clean = StandardScaler()

# # Fit the scaler on your data and transform it
# scaled_cleandata_2d = scaler_clean.fit_transform(clean_data_2d)

# # Reshape the scaled data back to its original shape
# scaled_clean_data = scaled_cleandata_2d.reshape(original_shape_clean)

In [13]:
# np.std(scaled_clean_data)

## Noisy

In [14]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_noisy']  # Replace with the actual variable name

# Define the root folder containing subfolders with .mat files
root_folder = 'data/dataformodel_noaug/'

# Specify subfolders of interest
subfolders = [
#     'noisy_stft_pink-5db',
#     'noisy_stft_pink0db',
#     'noisy_stft_pink5db',
    'noisy_stft_wgn5db',
    'noisy_stft_wgn-5db',
    'noisy_stft_wgn0db',
#     'noisy_stft_han',
#     'noisy_stft_red-5db_Hristo',
#     'noisy_stft_red0db_Hristo',
#     'noisy_stft_red5db_Hristo',
]

# Initialize an empty list to store loaded data
loaded_data = []

# Loop through each subfolder and load data
for subfolder in subfolders:
    mat_folder = os.path.join(root_folder, subfolder)

    # List all .mat files in the current subfolder
    mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

    # Use concurrent processing to load data from multiple files simultaneously
    with concurrent.futures.ThreadPoolExecutor() as executor:
        current_loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files), desc=f'Loading {subfolder}'))

    # Extend the loaded_data list with data from the current subfolder
    loaded_data.extend(current_loaded_data)

# Stack all loaded data into a single array
noisy_data = np.stack(loaded_data)

Loading noisy_stft_wgn5db: 4213it [00:02, 1827.89it/s]
Loading noisy_stft_wgn-5db: 4213it [00:02, 1678.53it/s]
Loading noisy_stft_wgn0db: 4213it [00:02, 1654.32it/s]


In [15]:
noisy_data.shape,np.std(np.real(noisy_data)),np.mean(np.real(noisy_data))

((12639, 128, 64), 0.9561850481069908, 7.963804402420294e-05)

In [7]:
# Function to load data from a single .mat file
def load_data(file_path):
    loaded_mat = scipy.io.loadmat(file_path)
    return loaded_mat['Segment_noisy']  # Replace with the actual variable name

# Define the folder containing the .mat file
mat_folder = 'data/dataformodel_noaug/noisy_stft_wgn-5db'  # Replace with the actual path

# Initialize an empty list to store loaded data
loaded_data = []

# List all .mat files in the folder
mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# Use concurrent processing to load data from multiple files simultaneously
with concurrent.futures.ThreadPoolExecutor() as executor:
    loaded_data = list(tqdm.tqdm(executor.map(load_data, mat_files)))

# Concatenate all loaded data into a single variable
noisy_data = np.stack(loaded_data, axis=0)

4213it [00:02, 1788.49it/s]


In [8]:
noisy_data

array([[[ 3.08636244e+00+0.j        ,  3.56211699e-01+0.j        ,
         -1.11889181e+00+0.j        , ...,  2.73568273e-01+0.j        ,
         -1.17926995e-01+0.j        , -1.39323283e+00+0.j        ],
        [ 7.78737269e-01-2.69618859j,  1.20977585e+00-0.1588504j ,
         -2.10388009e-01+1.06510527j, ..., -2.85339903e-01-0.33983802j,
          5.76239217e-01+0.03189136j,  1.86197689e-01+1.40566032j],
        [-1.79043790e+00-0.59086658j,  3.05546001e-01-2.06329935j,
          7.96781647e-01+0.54473782j, ..., -5.06345862e-01+0.56954715j,
         -1.12488510e-01-0.86575752j,  1.29363744e+00-0.42301794j],
        ...,
        [ 1.60480130e+00+2.44657477j,  8.08142396e-02-1.45825917j,
         -1.08379316e+00+0.24187403j, ...,  5.34948012e-01+0.19036978j,
          2.87079761e-01-0.84167076j, -2.94255465e-01-1.07491487j],
        [ 2.39544983e+00-1.06288208j, -8.60798681e-01-0.56385494j,
          7.32005218e-01+1.1418706j , ..., -3.69368694e-01-0.77702032j,
         -1.48933969

In [17]:
# noisy_data.shape,np.std(np.real(noisy_data)),np.mean(np.real(noisy_data))

In [18]:
noisy_real = np.real(noisy_data)
noisy_img  = np.imag(noisy_data)
noisy_real.shape,noisy_img.shape

((12639, 128, 64), (12639, 128, 64))

In [19]:
# original_shape = noisy_real.shape
# noisy_data_2d = noisy_real.reshape(-1, original_shape[-1])

# # Initialize the StandardScaler
# scaler = StandardScaler()

# # Fit the scaler on your data and transform it
# scaled_data_2d = scaler.fit_transform(noisy_data_2d)

# # Reshape the scaled data back to its original shape
# scaled_data = scaled_data_2d.reshape(original_shape)

In [20]:
# np.mean(scaled_data)

In [21]:
from sklearn.model_selection import train_test_split
X_trn, X_valid, y_trn, y_valid = train_test_split(noisy_real, clean_real, test_size=0.1, random_state=42)

In [22]:
X_trn.shape, X_valid.shape, y_trn.shape, y_valid.shape

((11375, 128, 64), (1264, 128, 64), (11375, 128, 64), (1264, 128, 64))

## Model Run

In [23]:
# # Compile the model (choose appropriate loss and optimizer)
# # Set the Adam optimizer with a custom learning rate

# initial_learning_rate = 0.0001  # Define your initial learning rate 0.0001  best
# adam_optimizer = Adam(learning_rate=initial_learning_rate)

# model.compile(loss='mean_squared_error', optimizer=adam_optimizer,metrics="accuracy")

# # Train the model
# model.fit(X_trn,y_trn , epochs=10000, batch_size=64,validation_data=(X_valid,y_valid))

In [24]:
from sklearn.model_selection import KFold


initial_learning_rate = 0.0001 # Define your initial learning rate
adam_optimizer = Adam(learning_rate=initial_learning_rate)


model.compile(loss='mean_squared_error', optimizer=adam_optimizer, metrics=['accuracy'])


# Define the number of folds for cross-validation
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True)

fold_counter = 1
# Train the model using cross-noisy_real
for train_index, test_index in kf.split(noisy_real):
    #print(train_index, test_index)
    
    X_train, X_test = noisy_real[train_index], noisy_real[test_index]
    y_train, y_test = clean_real[train_index], clean_real[test_index]
      
    print(f"Training Fold {fold_counter}")
    model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test))
    
    fold_counter += 1

Training Fold 1
Epoch 1/300
178/178 [==============================] - 33s 111ms/step - loss: 0.3775 - accuracy: 0.1014 - val_loss: 0.3834 - val_accuracy: 0.0842
Epoch 2/300
178/178 [==============================] - 17s 98ms/step - loss: 0.3375 - accuracy: 0.1014 - val_loss: 0.3405 - val_accuracy: 0.0842
Epoch 3/300
178/178 [==============================] - 17s 96ms/step - loss: 0.3035 - accuracy: 0.1014 - val_loss: 0.2882 - val_accuracy: 0.0842
Epoch 4/300
178/178 [==============================] - 18s 99ms/step - loss: 0.2751 - accuracy: 0.1014 - val_loss: 0.2495 - val_accuracy: 0.0842
Epoch 5/300
178/178 [==============================] - 17s 98ms/step - loss: 0.2546 - accuracy: 0.1014 - val_loss: 0.2393 - val_accuracy: 0.0842
Epoch 6/300
178/178 [==============================] - 18s 99ms/step - loss: 0.2375 - accuracy: 0.1014 - val_loss: 0.2242 - val_accuracy: 0.0842
Epoch 7/300
178/178 [==============================] - 18s 100ms/step - loss: 0.2251 - accuracy: 0.1014 - val_los

Epoch 57/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1274 - accuracy: 0.1014 - val_loss: 0.1249 - val_accuracy: 0.0842
Epoch 58/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1263 - accuracy: 0.1014 - val_loss: 0.1261 - val_accuracy: 0.0842
Epoch 59/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1260 - accuracy: 0.1014 - val_loss: 0.1231 - val_accuracy: 0.0842
Epoch 60/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1260 - accuracy: 0.1014 - val_loss: 0.1247 - val_accuracy: 0.0842
Epoch 61/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1257 - accuracy: 0.1014 - val_loss: 0.1241 - val_accuracy: 0.0842
Epoch 62/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1253 - accuracy: 0.1014 - val_loss: 0.1232 - val_accuracy: 0.0842
Epoch 63/300
178/178 [==============================] - 18s 101ms/step - loss: 0.1255 - accuracy: 0.1014 - val_loss: 0

Epoch 113/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1187 - accuracy: 0.1014 - val_loss: 0.1172 - val_accuracy: 0.0842
Epoch 114/300
178/178 [==============================] - 18s 101ms/step - loss: 0.1188 - accuracy: 0.1014 - val_loss: 0.1195 - val_accuracy: 0.0842
Epoch 115/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1186 - accuracy: 0.1014 - val_loss: 0.1170 - val_accuracy: 0.0842
Epoch 116/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1182 - accuracy: 0.1014 - val_loss: 0.1172 - val_accuracy: 0.0842
Epoch 117/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1181 - accuracy: 0.1014 - val_loss: 0.1175 - val_accuracy: 0.0842
Epoch 118/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1184 - accuracy: 0.1014 - val_loss: 0.1170 - val_accuracy: 0.0842
Epoch 119/300
178/178 [==============================] - 18s 101ms/step - loss: 0.1181 - accuracy: 0.1014 - val_

178/178 [==============================] - 18s 100ms/step - loss: 0.1132 - accuracy: 0.1014 - val_loss: 0.1137 - val_accuracy: 0.0842
Epoch 224/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1131 - accuracy: 0.1014 - val_loss: 0.1141 - val_accuracy: 0.0842
Epoch 225/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1130 - accuracy: 0.1014 - val_loss: 0.1134 - val_accuracy: 0.0842
Epoch 226/300
178/178 [==============================] - 18s 99ms/step - loss: 0.1129 - accuracy: 0.1014 - val_loss: 0.1137 - val_accuracy: 0.0842
Epoch 227/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1129 - accuracy: 0.1014 - val_loss: 0.1135 - val_accuracy: 0.0842
Epoch 228/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1129 - accuracy: 0.1014 - val_loss: 0.1133 - val_accuracy: 0.0842
Epoch 229/300
178/178 [==============================] - 18s 99ms/step - loss: 0.1129 - accuracy: 0.1014 - val_loss: 0.1136 - v

178/178 [==============================] - 18s 99ms/step - loss: 0.1099 - accuracy: 0.0989 - val_loss: 0.1071 - val_accuracy: 0.1061
Epoch 34/300
178/178 [==============================] - 18s 99ms/step - loss: 0.1099 - accuracy: 0.0989 - val_loss: 0.1071 - val_accuracy: 0.1061
Epoch 35/300
178/178 [==============================] - 18s 99ms/step - loss: 0.1099 - accuracy: 0.0989 - val_loss: 0.1073 - val_accuracy: 0.1061
Epoch 36/300
178/178 [==============================] - 18s 101ms/step - loss: 0.1099 - accuracy: 0.0989 - val_loss: 0.1072 - val_accuracy: 0.1061
Epoch 37/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1099 - accuracy: 0.0989 - val_loss: 0.1076 - val_accuracy: 0.1061
Epoch 38/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1101 - accuracy: 0.0989 - val_loss: 0.1079 - val_accuracy: 0.1061
Epoch 39/300
178/178 [==============================] - 18s 100ms/step - loss: 0.1101 - accuracy: 0.0989 - val_loss: 0.1072 - val_accu

...

178/178 [==============================] - 24s 132ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1062 - val_accuracy: 0.1050
Epoch 199/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1062 - val_accuracy: 0.1050
Epoch 200/300
178/178 [==============================] - 23s 132ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1063 - val_accuracy: 0.1050
Epoch 201/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1063 - val_accuracy: 0.1050
Epoch 202/300
178/178 [==============================] - 23s 132ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1062 - val_accuracy: 0.1050
Epoch 203/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1063 - val_accuracy: 0.1050
Epoch 204/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1064 - accuracy: 0.0990 - val_loss: 0.1062 -

178/178 [==============================] - 23s 132ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 9/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 10/300
178/178 [==============================] - 24s 132ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 11/300
178/178 [==============================] - 23s 130ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 12/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 13/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_accuracy: 0.0928
Epoch 14/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1105 - val_ac

Epoch 64/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1106 - val_accuracy: 0.0928
Epoch 65/300
178/178 [==============================] - 23s 130ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1107 - val_accuracy: 0.0928
Epoch 66/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1106 - val_accuracy: 0.0928
Epoch 67/300
178/178 [==============================] - 23s 132ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1106 - val_accuracy: 0.0928
Epoch 68/300
178/178 [==============================] - 23s 131ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1106 - val_accuracy: 0.0928
Epoch 69/300
178/178 [==============================] - 23s 130ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1106 - val_accuracy: 0.0928
Epoch 70/300
178/178 [==============================] - 23s 129ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0

178/178 [==============================] - 22s 126ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 175/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 176/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 177/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1059 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 178/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 179/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1109 - val_accuracy: 0.0928
Epoch 180/300
178/178 [==============================] - 22s 126ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1109 -

178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 285/300
178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 286/300
178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 287/300
178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 288/300
178/178 [==============================] - 23s 129ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 289/300
178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 - val_accuracy: 0.0928
Epoch 290/300
178/178 [==============================] - 23s 128ms/step - loss: 0.1058 - accuracy: 0.1004 - val_loss: 0.1111 -

In [25]:
model.save("all_wgn_attnunetresi.h5")

C:\Users\Admin\.conda\envs\cuda_tf\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
aabgh

In [23]:
# from sklearn.model_selection import KFold

# si_sdr = ScaleInvariantSignalDistortionRatio()


# initial_learning_rate = 0.001  # Define your initial learning rate
# adam_optimizer = Adam(learning_rate=initial_learning_rate)

# model.compile(loss='mean_squared_error', optimizer=adam_optimizer, metrics=['accuracy'])

# # Define the number of folds for cross-validation
# num_folds = 10
# kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# fold_counter = 1
# sisdr_values = []  # List to store SNR values for each fold

# # Train the model using cross-validation
# for train_index, test_index in kf.split(noisy_real):
#     X_train, X_test = noisy_real[train_index], noisy_real[test_index]
#     y_train, y_test = clean_real[train_index], clean_real[test_index]

#     print(f"Training Fold {fold_counter}")
#     model.fit(X_train, y_train, epochs=150, batch_size=64, validation_data=(X_test, y_test))

#     # Make predictions
#     pred = model.predict(X_test)
    
#     target = tensor([np.expand_dims(y_test,axis=-1)])
#     preds = tensor([pred])

#     # Calculate SNR
#     sisdr = si_sdr(preds, target)
    
#     print(f"sisdr for Fold {fold_counter}: {sisdr} dB")
    
#     sisdr_values.append(sisdr)  # Store SNR value for the current fold

#     fold_counter += 1

# # Calculate the average SNR across all folds
# final_sisdr = np.mean(sisdr_values)
# print(f"Final sisdr across all folds: {final_sisdr} dB")

In [24]:
aaaaacvfsv

NameError: name 'aaaaacvfsv' is not defined

In [25]:
pred = model.predict(X_valid)

In [26]:
# pred = model.predict(X_valid)
10 * np.log10(np.sum(np.expand_dims(y_valid, axis=-1)**2) / np.sum((np.expand_dims(y_valid, axis=-1) - pred)**2))

8.340421022458738

In [23]:
# mat_folder

'data/dataformodel_noaug/noisy_stft_han'

In [1]:
# si_sdr = ScaleInvariantSignalDistortionRatio()
# target = tensor([np.expand_dims(y_valid,axis=-1)])
# preds = tensor([pred])
# si_sdr(preds, target)

In [29]:
# model.save("unet_attn_wgn0.h5")

C:\Users\Admin\.conda\envs\cuda_tf\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [28]:
# p = model.predict(np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1))
# p= p.reshape((128, 64))
# q = p - noisy_data_real
# scipy.io.savemat('New_AS_002_denoise_model0001_aug_cv250_gg.mat', {'denoise_spe':q})

# test unseen data

In [ ]:
# # Function to load data from a single .mat file
# def load_data(file_path):
#     loaded_mat = scipy.io.loadmat(file_path)
#     return loaded_mat['Segment_clean']  # Replace with the actual variable name

# # Define the folder containing the .mat files
# mat_folder = 'data/unseenfordenoise/test_clean_stft_aug_wgn-5db'  # Replace with the actual path

# # Initialize an empty list to store loaded data
# loaded_test_data = []

# # List all .mat files in the folder
# mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# # Use concurrent processing to load data from multiple files simultaneously
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     loaded_test_data = list(executor.map(load_data, mat_files))

# # Concatenate all loaded data into a single variable
# test_clean_data = np.stack(loaded_test_data, axis=0)

In [ ]:
# test_clean_real = np.real(test_clean_data)
# test_clean_img  = np.imag(test_clean_data)
# test_clean_real.shape,test_clean_img.shape

In [ ]:
# # Function to load data from a single .mat file
# def load_data(file_path):
#     loaded_mat = scipy.io.loadmat(file_path)
#     return loaded_mat['Segment_noisy']  # Replace with the actual variable name

# # Define the folder containing the .mat file
# mat_folder = 'data/unseenfordenoise/test_noisy_stft_aug_wgn-5db'  # Replace with the actual path

# # Initialize an empty list to store loaded data
# loaded_test_data = []

# # List all .mat files in the folder
# mat_files = [os.path.join(mat_folder, filename) for filename in os.listdir(mat_folder) if filename.endswith('.mat')]

# # Use concurrent processing to load data from multiple files simultaneously
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     loaded_test_data = list(executor.map(load_data, mat_files))

# # Concatenate all loaded data into a single variable
# test_noisy_data = np.stack(loaded_test_data, axis=0)

In [ ]:
# test_noisy_real = np.real(test_noisy_data)
# test_noisy_img  = np.imag(test_noisy_data)
# test_noisy_real.shape,test_noisy_img.shape

In [ ]:
# pred = model.predict(test_noisy_real)
# 10 * np.log10(np.sum(np.expand_dims(test_clean_real, axis=-1)**2) / np.sum((np.expand_dims(test_clean_real, axis=-1) - pred)**2))

In [ ]:
# si_sdr = ScaleInvariantSignalDistortionRatio()
# target = tensor([np.expand_dims(test_clean_real,axis=-1)])
# preds = tensor([pred])
# si_sdr(preds, target)

In [22]:
model.save('denoisy_attnunet_0dbwg.h5')

C:\Users\Admin\.conda\envs\cuda_tf\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


## Prediction

In [2]:
from keras.models import load_model

# Load the model from the file
modell = load_model("model\mix_model_attn.h5")

In [6]:
# Create an empty list to store the denoised spectrograms
denoised_spectrograms = []

# Specify the directory containing the noisy data files
noisy_data_directory = 'test_noisy_data/'

# Loop through all files in the directory
for file_name in os.listdir(noisy_data_directory):
    if file_name.endswith('.mat'):
        # Load the noisy data from the MAT file
        file_path = os.path.join(noisy_data_directory, file_name)
        test_noisy_data = scipy.io.loadmat(file_path)['Segment_noisy']

        noisy_data_real = np.real(test_noisy_data)
        noisy_data_imag = np.imag(test_noisy_data)
        
        
        # Expand dimensions
        noisy_data_real = np.expand_dims(noisy_data_real[np.newaxis, :, :], axis=-1)

        # Apply the model to denoise the data
        denoised_data_real = modell.predict(noisy_data_real)

        denoised_data_real = denoised_data_real.reshape((128, 64))
        
        # Reshape to (128, 64)
        #denoised_data = np.vectorize(complex)(denoised_data_real, noisy_data_imag)
        

        # Append the denoised data to the list
        denoised_spectrograms.append(denoised_data_real)

# Stack the denoised spectrograms along a new axis (4D array)
merged_denoised_data = np.concatenate(denoised_spectrograms, axis=1)

1/1 [==============================] - 0s 36ms/step


In [24]:
merged_denoised_data.shape

(128, 320)

In [25]:
scipy.io.savemat('New_AS_002_denoise_model0001_aug_cv250.mat', {'denoise_spe': merged_denoised_data})

In [26]:
merged_denoised_data

array([[-2.3229254e-02,  6.2912568e-02,  1.4227118e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.8272759e-01,  3.0651870e-01, -8.8779166e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 7.1703506e-01, -1.1901593e+00,  3.0767056e-01, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 2.4114050e-05,  1.0826528e-03, -9.6957019e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-3.7938808e-05, -8.5382385e-04, -4.6788628e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-5.4672104e-04, -1.1342473e-03,  2.0025505e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)